In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

In [21]:
text_x_src_path = '../../Dataset/mixed_text_assignment.json'
numb_x_src_path = '../../Dataset/MixedDataSet.json'
y_src_path = '../../DataBook/Mixed_Data_Analyst.xlsx'

In [22]:
text_data = pd.read_json(text_x_src_path)
numb_data = pd.read_json(numb_x_src_path)

In [23]:
df_supervision = pd.read_excel(y_src_path)
plagiarised_array = df_supervision['Plagiarised'].astype(int).values

In [24]:
texts = text_data.iloc[:, :].astype(str).values.tolist()

texts = [[element if element != 'None' else '' for element in sublist] for sublist in texts]

In [25]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>") 
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

max_sequence_len = max([len(seq) for seq in sequences])
data_vectorized = pad_sequences(sequences, maxlen=max_sequence_len)

In [26]:
X_T = data_vectorized
X_N = np.nan_to_num(numb_data.values, nan=0, copy=True).astype(int)
y = plagiarised_array

In [27]:
X_T_train, X_T_test, y_T_train, y_T_test = train_test_split(X_T, y, test_size=0.2, random_state=32)

X_N_train, X_N_test, y_N_train, y_N_test = train_test_split(X_N, y, test_size=0.2, random_state=32)

In [28]:
textP = Perceptron()
textP.fit(X_T_train,y_T_train)

print(f"Text Training data score: {textP.score(X_T_train, y_T_train)}")
print(f"Text Test data score: {textP.score(X_T_test, y_T_test)}")

Text Training data score: 1.0
Text Test data score: 0.8888888888888888


In [29]:
numbP = Perceptron()
numbP.fit(X_N_train,y_N_train)

print(f"Text Training data score: {numbP.score(X_N_train, y_N_train)}")
print(f"Text Test data score: {numbP.score(X_N_test, y_N_test)}")

Text Training data score: 1.0
Text Test data score: 0.8518518518518519


In [30]:
text_model = keras.models.Sequential([
    keras.layers.Input(shape=(10315,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

LOSS_FN = keras.losses.BinaryCrossentropy()

text_model.compile(optimizer='adam', loss=LOSS_FN, metrics=['accuracy'])

text_model.fit(X_T_train, y_T_train, epochs=5, validation_data=(X_T_test, y_T_test))

y_pred_text_binary_flat = (text_model.predict(X_T_test).flatten() > 0.5).astype(int)



Epoch 1/5


4/4 [==============================] - 1s 58ms/step - loss: 7.5901 - accuracy: 0.6509 - val_loss: 2.3365 - val_accuracy: 0.7778
Epoch 2/5
4/4 [==============================] - 0s 11ms/step - loss: 1.0887 - accuracy: 0.8962 - val_loss: 1.5957 - val_accuracy: 0.7778
Epoch 3/5
4/4 [==============================] - 0s 11ms/step - loss: 0.8148 - accuracy: 0.9434 - val_loss: 2.2705 - val_accuracy: 0.6296
Epoch 4/5
4/4 [==============================] - 0s 11ms/step - loss: 0.2698 - accuracy: 0.9151 - val_loss: 1.1196 - val_accuracy: 0.7778
Epoch 5/5
1/1 [==============================] - 0s 66ms/step


In [31]:
numb_model = keras.models.Sequential([
    keras.layers.Input(shape=(75711,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

LOSS_FN = keras.losses.BinaryCrossentropy()

numb_model.compile(optimizer='adam', loss=LOSS_FN, metrics=['accuracy'])

numb_model.fit(X_N_train, y_N_train, epochs=5, validation_data=(X_N, y))

numb_model.save("numb_model.h5")

y_pred_numb_binary_flat = (numb_model.predict(X_N_test).flatten() > 0.5).astype(int)

Epoch 1/5
4/4 [==============================] - 1s 69ms/step - loss: 11654.0420 - accuracy: 0.7736 - val_loss: 5816.8003 - val_accuracy: 0.8872
Epoch 2/5
4/4 [==============================] - 0s 31ms/step - loss: 4279.8164 - accuracy: 0.8396 - val_loss: 772.7741 - val_accuracy: 0.5865
Epoch 3/5
4/4 [==============================] - 0s 29ms/step - loss: 591.8410 - accuracy: 0.3868 - val_loss: 0.6433 - val_accuracy: 0.1729
Epoch 4/5
4/4 [==============================] - 0s 27ms/step - loss: 0.6537 - accuracy: 0.6415 - val_loss: 0.6418 - val_accuracy: 0.8947
Epoch 5/5
1/1 [==============================] - 0s 41ms/step


c:\Users\K-Gen\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
for numb, text, actual in zip(y_pred_numb_binary_flat, y_pred_text_binary_flat, y_N_test):
    print(f"Number: {numb}, Textual: {text} Actual: {actual}")

Number: 0, Textual: 0 Actual: 0
Number: 1, Textual: 1 Actual: 1
Number: 1, Textual: 0 Actual: 1
Number: 0, Textual: 0 Actual: 1
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 1, Textual: 0 Actual: 1
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 1, Textual: 0 Actual: 1
Number: 1, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0


In [33]:
print(text_model.evaluate(X_T_test,y_T_test)[1])
print(numb_model.evaluate(X_N_test,y_N_test)[1])

1/1 [==============================] - 0s 20ms/step - loss: 1.0357 - accuracy: 0.8519
0.8518518805503845
1/1 [==============================] - 0s 24ms/step - loss: 0.5957 - accuracy: 0.9259
0.9259259104728699
